In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.append("../")

from src.training_utils.training import load_model
from src.line_detection.hough import *
from ast import literal_eval
from src.ocr_utils.ocr import *

In [16]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from IPython.display import display
from PIL import Image
from torchvision import transforms as T
import torchvision
from paddleocr import PaddleOCR

In [4]:
PATH_MODELS = "/home/nacho/TFI-Cazcarra/data/models"
PATH_YOLO = "/home/nacho/TFI-Cazcarra/yolov3/runs/train"

## Realizo predicciones

In [5]:
img_path = '/home/nacho/TFI-Cazcarra/data/imagenes_diagramas/ERDiagramsMySQL-5.png'
img = Image.open(img_path).convert("RGB")
transform = T.Compose([T.ToTensor()])
img_tensor = transform(img)

In [ ]:
def draw_bbox(img, xmin, ymin, xmax, ymax, score, label): 
    txt = get_class_name(label, le_dict) + ' ' + str(score)
    img = cv2.putText(img, txt, (int(xmin), int(ymin)),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255), 1)

    return cv2.rectangle(img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), 
                         (255,0,0), 1)

### 1. Extraigo las tablas

In [ ]:
retinanet_tablas = load_model(os.path.join(PATH_MODELS, "model_best_tablas_retinanet.pt"))
faster_rcnn_tablas = load_model(os.path.join(PATH_MODELS, "model_best_tablas_fasterrcnn.pt"))

In [ ]:
yolo_tablas = torch.hub.load('ultralytics/yolov5', 'custom', \
                             os.path.join(PATH_YOLO, "exp5", "weights", "best.pt"))

In [ ]:
tablas_1 = retinanet_tablas([img_tensor])
valid_idx_1 = torchvision.ops.nms(tablas_1[1][0]['boxes'], tablas_1[1][0]['scores'], 0.1)

coords_1 = tablas_1[1][0]['boxes'][valid_idx_1]
scores_1 = tablas_1[1][1]['scores'][valid_idx_1]

In [ ]:
tablas_2 = faster_rcnn_tablas([img_tensor])
valid_idx_2 = torchvision.ops.nms(tablas_2[1][0]['boxes'], tablas_2[1][0]['scores'], 0.1)

coords_2 = tablas_2[1][0]['boxes'][valid_idx_1]
scores_2 = tablas_2[1][1]['scores'][valid_idx_1]

In [ ]:
tablas_3 = yolo_tablas(Image.open('/home/nacho/TFI-Cazcarra/data/imagenes_diagramas/ERDiagramsMySQL-1.png').convert("RGB"))

#x1 y1 x2 y2 score class
coords_3 = tablas_3.xyxyn[0][:, :4]
scores_3 = tablas_3.xyxyn[0][:, 4]

valid_idx_3 = torchvision.ops.nms(coords_3, scores_3, 0.1)

# Falta filtrar acá

### 2. Extraigo las cardinalidades

In [ ]:
retinanet_tablas = load_model(os.path.join(PATH_MODELS, "model_best_cardinalidades_retinanet.pt"))
faster_rcnn_tablas = load_model(os.path.join(PATH_MODELS, "model_best_cardinalidades_fasterrcnn.pt"))

yolo_tablas = torch.hub.load('ultralytics/yolov5', 'custom', \
                             os.path.join(PATH_YOLO, "exp3", "weights", "best.pt"))

In [ ]:
## TODO: Dividir la imagen en frames y etc etc.

## Encuentro las líneas y junto con las cardinalidades

In [6]:
OFFSET_TABLAS = 5

In [7]:
def find_lines(img_path):
    img_basename = os.path.basename(img_path)
    tablas, cardinalidades = get_tablas(img_basename)
    #Agrandamos un toque las cajas
    offset = np.array([-OFFSET_TABLAS, -OFFSET_TABLAS, OFFSET_TABLAS, OFFSET_TABLAS]).reshape(1,4)
    tablas = np.sum([tablas, offset])

    #No le paso las cardinalidades.
    img, all_lines = apply_hough(img_path, tablas, [])

    all_points = lines_to_points(all_lines)
    lines = hough_detecting(all_points)

    return unify_cardinalidades(img, lines, cardinalidades)

In [8]:
def line_to_points(x, y, max_dst_per_points=2):
    dst_line = math.dist(x,y)
    parts = dst_line // max_dst_per_points
    return np.linspace(x, y, int(parts)+1).tolist()


def dist_func(comb):
    return math.dist(comb[0], comb[1])


def get_centroid(cardinalidad):
    center_x  = cardinalidad[0] + (cardinalidad[2] - cardinalidad[0])/2
    center_y  = cardinalidad[1] + (cardinalidad[3] - cardinalidad[1])/2
    return (center_x, center_y)

def nearest_tabla_from_cardinalidad(cardinalidad, tablas):
    # Dadas las coords de una cardinalidad, devolver a qué tabla está más cerca.
    # TODO: Puedo mezclarlo con el de OCR y reemplazar las coordenadas por el nombre de la
    # tabla... asi se entiende mejor. Esto quizás no hace falta pero no estaría mal.
    cardinalidad = literal_eval(cardinalidad)
    cardinalidad_centroid = get_centroid(cardinalidad)
    dict_dist_tablas = {}
    for t in tablas:
        tabla_points = []
        # De x1y1 a x2y1
        tabla_points += line_to_points((t[0], t[1]), (t[2], t[1]))
        # De x1y1 a x1y2
        tabla_points += line_to_points((t[0], t[1]), (t[0], t[3]))
        # De x1y2 a x2y2
        tabla_points += line_to_points((t[0], t[3]), (t[2], t[3]))
        # De x2y1 a x2y2
        tabla_points += line_to_points((t[2], t[1]), (t[2], t[3]))
        dist_cardinalidad = [(cardinalidad_centroid, point) for point in tabla_points]
        min_combination = min(dist_cardinalidad, key=dist_func)
        dict_dist_tablas[",".join([str(c) for c in t])] = dist_func(min_combination)
    nearest_tabla = min(dict_dist_tablas, key=dict_dist_tablas.get)
    return [int(c) for c in nearest_tabla.split(",")]

In [9]:
pairs = []

img_basename = os.path.basename(img_path)
tablas, _ = get_tablas(img_basename)

for line_name, line in find_lines(img_path).items():
    cardinalidades = line.split("|")
    tabla_a = nearest_tabla_from_cardinalidad(cardinalidades[0], tablas)
    tabla_b = nearest_tabla_from_cardinalidad(cardinalidades[1], tablas)
    pairs.append((tabla_a, tabla_b))

In [10]:
pairs

[([289, 16, 440, 150], [18, 89, 211, 344]),
 ([291, 191, 449, 383], [18, 89, 211, 344]),
 ([291, 191, 449, 383], [516, 241, 664, 334])]

## OCR

In [11]:
# Una vez que tenemos la lista de tuplas con las conexiones de las tablas, podemos sacar
# las FK y de ahi, descartar los demás y poder sacar la/s PK.

In [13]:
# TODO: Choose algo and language.
ocr = PaddleOCR(use_angle_cls=False, lang='en', show_log=False, 
                det_algorithm_dir="/home/nacho/TFI-Cazcarra/ocr/db/", 
                rec_algorithm_dir="/home/nacho/TFI-Cazcarra/ocr/svtr/", det_db_score_mode="slow") 

In [22]:
coords = []
results = []

img_arr = np.array(img)

for t in tablas:
    tabla_cropped = img_arr[t[1]:t[3], t[0]:t[2]]
    tabla_cropped = reescale(tabla_cropped)
    if len(tabla_cropped.shape) == 3 and tabla_cropped.shape[-1] > 3:
        tabla_cropped = tabla_cropped[:,:,:3]
    result = ocr.ocr(tabla_cropped, cls=False)
    
    coords.append(t.tolist())
    results.append(result[0])

In [25]:
all_tables = {}
tables_names = {}

for c, r in zip(coords, results):
    boxes = [line[0] for line in r]
    txts = [line[1][0].strip() for line in r]
    scores = [line[1][1] for line in r]
    table, dict_attributes = clean_texts(txts)
    all_tables[table] = dict_attributes
    tables_names[table] = c

In [41]:
def pairs_to_names(pairs, tables_names):
    new_pairs = []
    tables_names = {str(v):k for k,v in tables_names.items()}
    
    for pair in pairs:
        tabla_a, tabla_b = pair
        tabla_a, tabla_b = tables_names[str(tabla_a)], tables_names[str(tabla_b)]
        new_pairs.append((tabla_a, tabla_b))
    return new_pairs

In [42]:
pairs_to_names(pairs, tables_names)

[('tokens', 'users'), ('poems', 'users'), ('poems', 'categories')]

In [51]:
def extract_candidate_keys(table_attributes):
    for att in table_attributes:
        if "_id" in att:
            print(att)

In [53]:
## TODO: En base a esto construir las FK y las PK. Una vez hecho eso, también definir las many to many.
for k in all_tables.keys():  
    extract_candidate_keys(all_tables[k].keys())

user_id
poem_id
category_id
user_id
token_id
user_id
category_id


## Una vez que está todo hecho, unificar acá y detectar puntos de mejora.